In [1]:
import os
import random
import math
import re
import collections
import copy

In [2]:
MAX_WORD_LEN = 30

In [3]:
class DictPruner:
    """Base class for pruning a dictionary of words that are no longer relevant for the search"""
    def prune(self, word, dictionary):
        return dictionary

In [4]:
class LettersExistPruner(DictPruner):
    """Class for pruning words from dictionary that do not contain all of the revealed letters from the word"""
    def prune(self, word, dictionary):
        letters = set(word.replace(".", ""))            
        new_dictionary = []
        for dict_word in dictionary:
            if letters.issubset(set(dict_word)):
                new_dictionary.append(dict_word)
        return new_dictionary

In [5]:
class ExactMatchPruner(DictPruner):
    """Class for pruning words from dictionary that do not exactly match the word"""
    def prune(self, word, dictionary):
        new_dictionary = []
        pattern = re.compile(word)
        for dict_word in dictionary:
            if re.fullmatch(pattern, dict_word):
                new_dictionary.append(dict_word)
        return new_dictionary

In [6]:
class PrefixSuffixPruner(DictPruner):
    """Class for pruning words from dictionary that do not have word as prefix or suffix"""
    def prune(self, word, dictionary):
        new_dictionary = []
        pattern = re.compile(f"^{word}|{word}$")
        for dict_word in dictionary:
            if pattern.search(dict_word):
                new_dictionary.append(dict_word)
        return new_dictionary

In [7]:
class MissingLettersPruner(DictPruner):
    """Class for pruning words from dictionary that have any of the already guessed letters"""
    def __init__(self, guessed_letters):
        self.guessed_letters = guessed_letters

    def prune(self, word, dictionary):
        bad_letters = set(self.guessed_letters) - set(word.replace(".", ""))
        new_dictionary = []
        for dict_word in dictionary:
            if bad_letters.isdisjoint(set(dict_word)):
                new_dictionary.append(dict_word)
        return new_dictionary            

In [8]:
class Guesser:
    def __init__(self, dictionary, pruners=[], name=None):
        self.full_dictionary = dictionary
        self.dictionary = copy.copy(self.full_dictionary)
        self.pruners = pruners
        self.name = name

    def reset(self):
        if len(self.dictionary) != len(self.full_dictionary):
            self.dictionary = copy.copy(self.full_dictionary)

    def rename(self, name):
        self.name = name

    def prune(self, word):
        for pruner in self.pruners:
            self.dictionary = pruner.prune(word, self.dictionary)

    def guess_letters(self, word):
        self.prune(word)
        letters_in_words = ["".join(set(dict_word)) for dict_word in self.dictionary]
        candidates = collections.Counter("".join(letters_in_words)).most_common()        
        return candidates

In [9]:
class Matcher(Guesser):
    def __init__(self, dictionary, size, pruners=[], name=None):
        self.size = size
        self.pruners = pruners
        self.name = name
        self.full_dictionary = self.get_dictionary(dictionary)
        self.dictionary = copy.copy(self.full_dictionary)

    def get_dictionary(self, dictionary):
        return dictionary

In [10]:
class PrefixMatcher(Matcher):
    def get_dictionary(self, dictionary):
        new_dictionary = []
        for dict_word in dictionary:
            sliced_word = dict_word[:self.size]
            if len(sliced_word) == self.size:
                new_dictionary.append(sliced_word)
        return new_dictionary

In [11]:
class SuffixMatcher(Matcher):
    def get_dictionary(self, dictionary):
        new_dictionary = []
        for dict_word in dictionary:
            sliced_word = dict_word[-self.size:]
            if len(sliced_word) == self.size:
                new_dictionary.append(sliced_word)
        return new_dictionary

In [12]:
def read(file):
    f = open(file, 'r', encoding='utf-8')
    data = f.read().splitlines()
    f.close()
    return data

In [13]:
train_data = read("train_data.txt")
val_data = read("val_data.txt")

guessed_letters = []

prefix_matchers = {}
suffix_matchers = {}
for size in range(MAX_WORD_LEN, 2, -1):
    prefix_matchers[size] = PrefixMatcher(train_data, size, pruners=[ExactMatchPruner(), MissingLettersPruner(guessed_letters)], name=f"{size}:PrefixMatcher")
    suffix_matchers[size] = SuffixMatcher(train_data, size, pruners=[ExactMatchPruner(), MissingLettersPruner(guessed_letters)], name=f"{size}:SuffixMatcher")

guessers = []
guessers.append(Guesser(train_data, pruners=[ExactMatchPruner(), MissingLettersPruner(guessed_letters)], name="ExactMatch"))
guessers.append(Guesser(train_data, pruners=[ExactMatchPruner()], name="ExactMatchWithMissingLetters"))
guessers.append(Guesser(train_data, pruners=[LettersExistPruner()], name="LettersExist"))
guessers.append(Guesser(train_data, [DictPruner()], name='Default'))

In [14]:
class Hangman:
    def __init__(self, train_data, val_data, seed=None):
        self.train_data = train_data
        self.val_data = val_data
        self.guessed_letters = guessed_letters
        freqs = collections.Counter("".join(["".join(set(word)) for word in self.train_data])).most_common()
        self.freq_order = {freqs[i][0]: i for i in range(len(freqs))}
        self.prefix_matchers = prefix_matchers
        self.suffix_matchers = suffix_matchers
        self.guessers = guessers
        if seed != None:
            random.seed(seed)
        

    def reset(self):
        self.guessed_letters.clear()
        for size, matcher in self.prefix_matchers.items():
            matcher.reset()
        for size, matcher in self.suffix_matchers.items():
            matcher.reset()
        for guesser in self.guessers:
            guesser.reset()
        
    
    def print(self, text, verbose):
        if verbose:
            print(text)
            

    def play(self, answer=None, verbose=False, version=0):
        self.reset()
        guess = [self.guess][version]
        if answer == None:
            answer = random.choice(self.val_data)
        tries_left = 6
        word = "."*len(answer)
        while tries_left > 0:
            guessed_letter = guess(word, tries_left, verbose)
            self.guessed_letters.append(guessed_letter)
            self.print(f"Guessed letter: {guessed_letter}", verbose)
            if guessed_letter not in answer:
                tries_left -= 1
                self.print(f"Letter not in word; Tries Left: {tries_left}", verbose)
            else:
                word = "".join([word[i] if answer[i] != guessed_letter else guessed_letter for i in range(len(word))])
                self.print(f"Letter found in word; New word: {word}", verbose)

            if word == answer:
                self.print("Word found!", verbose)
                return True
        self.print(f"Game Over; The word was: {answer}", verbose)
        return False

    
    def guess_letter(self, guesser, word, verbose=False):
        candidates = guesser.guess_letters(word)
        candidates.sort(key=lambda item: (item[1], -self.freq_order[item[0]]), reverse=True)
        for candidate, value in candidates:
            if candidate not in self.guessed_letters:
                self.print(f"{guesser.name}", verbose)
                return candidate

    def combine_candidates(self, candidates_list, size, verbose=False):
        combined = collections.defaultdict(int)
        for candidates in candidates_list:
            for candidate, count in candidates:
                if candidate in self.guessed_letters:
                    continue
                combined[candidate] += count

        candidates = sorted(combined.items(), key=lambda item: (item[1], -self.freq_order[item[0]]), reverse=True)
        if candidates != []:
            self.print(f"{size}-Matcher: {candidates[:5]}", verbose)
            return candidates[0][0]

    
    def guess(self, word, x, verbose=False):
        guessed_letter = None

        for size in range(len(word), 2, -1):
            if guessed_letter != None:
                return guessed_letter
            candidates_list = []
            prefix = word[:size]
            if prefix.count(".") <= size//2 and (size > 3 or prefix[0] != "."):
                candidates_list.append(self.prefix_matchers[size].guess_letters(prefix))
            suffix = word[-size:]
            if suffix.count(".") <= size//2 and (size > 3 or suffix[-1] != "."):
                candidates_list.append(self.suffix_matchers[size].guess_letters(suffix))
            guessed_letter = self.combine_candidates(candidates_list, size, verbose)

        for guesser in self.guessers:
            if guessed_letter != None:
                break
            guessed_letter = self.guess_letter(guesser, word, verbose)
        
        return guessed_letter

In [15]:
def get_win_rates(model, total_games=10**4):
    num_wins = 0
    num_games = 0
    
    for _ in range(total_games):
        num_games += 1
        if model.play():
            num_wins += 1
        if (_+1) % max(1, (total_games//10)) == 0 or (_+1) == total_games:
            print(f"Step {_+1}: Win Rate={num_wins/num_games*100:.2f}%")
    
    p = num_wins/num_games
    var = p*(1 - p)/num_games
    std = math.sqrt(var)
    print()
    print(f"Won {int(num_wins)} out of {int(num_games)} games; E[p]={p:.4f}, Var(p)={var:.4f}, std(p)={std:.4f}")
    print(f"Win Rate between {(p - std)*100:.2f}% and {(p + std)*100:.2f}% with 65% probability")
    print(f"Win Rate between {(p - 2*std)*100:.2f}% and {(p + 2*std)*100:.2f}% with 95% probability")
    print(f"Win Rate between {(p - 3*std)*100:.2f}% and {(p + 3*std)*100:.2f}% with 99.7% probability")

In [16]:
print("Training dataset...")
train_hangman = Hangman(train_data, train_data)
get_win_rates(train_hangman, 6400)

Training dataset...
Step 640: Win Rate=84.53%
Step 1280: Win Rate=86.56%
Step 1920: Win Rate=86.77%
Step 2560: Win Rate=86.56%
Step 3200: Win Rate=86.56%
Step 3840: Win Rate=86.56%
Step 4480: Win Rate=86.76%
Step 5120: Win Rate=86.62%
Step 5760: Win Rate=86.88%
Step 6400: Win Rate=86.77%

Won 5553 out of 6400 games; E[p]=0.8677, Var(p)=0.0000, std(p)=0.0042
Win Rate between 86.34% and 87.19% with 65% probability
Win Rate between 85.92% and 87.61% with 95% probability
Win Rate between 85.49% and 88.04% with 99.7% probability


In [17]:
print("Validation dataset...")
val_hangman = Hangman(train_data, val_data)
get_win_rates(val_hangman, 6400)

Validation dataset...
Step 640: Win Rate=60.78%
Step 1280: Win Rate=59.61%
Step 1920: Win Rate=59.64%
Step 2560: Win Rate=59.41%
Step 3200: Win Rate=60.16%
Step 3840: Win Rate=60.03%
Step 4480: Win Rate=59.91%
Step 5120: Win Rate=59.61%
Step 5760: Win Rate=60.03%
Step 6400: Win Rate=60.09%

Won 3846 out of 6400 games; E[p]=0.6009, Var(p)=0.0000, std(p)=0.0061
Win Rate between 59.48% and 60.71% with 65% probability
Win Rate between 58.87% and 61.32% with 95% probability
Win Rate between 58.26% and 61.93% with 99.7% probability
